<a href="https://colab.research.google.com/github/bencweems/gerrymandering/blob/master/gerrymandering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import pandas as pd

In [ ]:
all_data = pd.read_csv('1976-2018-house2.csv')

In [ ]:
results_2018 = all_data[all_data['year'] == 2018]

In [ ]:
results_2018_dr = results_2018[(results_2018.party == 'democrat') | (results_2018.party == 'republican')]

In [ ]:
results_2018_dr = results_2018_dr[['year', 'state', 'district', 'party', 'candidatevotes', 'totalvotes']].copy()

In [ ]:
def get_pct(row, party):
    if row['party'] == party and row['totalvotes']:
        return 100 * row['candidatevotes'] / row['totalvotes']
    return 0

In [ ]:
results_2018_pct = results_2018_dr[['year', 'state', 'district', 'dem_pct', 'rep_pct']].copy()

In [ ]:
results_2018_pct = results_2018_pct.groupby(by=['year', 'state', 'district']).sum()

In [ ]:
results_2018_pct

dem_pct    rep_pct
year state     district                      
2018 Alabama   1         36.776483  63.156333
               2         38.425938  61.388410
               3         36.218442  63.717310
               4         20.129108  79.774775
               5         38.894707  61.020129
...                            ...        ...
     Wisconsin 5         37.987801  61.934239
               6         44.463723  55.469214
               7         38.504213  60.110581
               8         36.275679  63.694209
     Wyoming   0         29.766205  63.585679

[435 rows x 2 columns]

In [ ]:
def get_pct_results(data, year):
    results_2018 = data[data['year'] == year]
    results_2018_dr = results_2018[(results_2018.party == 'democrat') | (results_2018.party == 'republican')]
    results_2018_dr_proj = results_2018_dr[['year', 'state', 'district', 'party', 'candidatevotes', 'totalvotes']].copy()
    results_2018_dr_proj['dem_pct'] = results_2018_dr_proj.apply(lambda row : get_pct(row, 'democrat'), axis=1)
    results_2018_dr_proj['rep_pct'] = results_2018_dr_proj.apply(lambda row : get_pct(row, 'republican'), axis=1)
    d_rows = results
    results_2018_pct_proj = results_2018_dr_proj[['year', 'state', 'district', 'dem_pct', 'rep_pct']].copy()
    results_2018_pct = results_2018_pct_proj.groupby(by=['year', 'state', 'district']).sum()
    results_2018_pct['rep_margin'] = results_2018_pct['rep_pct'] - results_2018_pct['dem_pct']
    return results_2018_pct, 

In [ ]:
from collections import defaultdict

results = defaultdict(dict)
for year in range(1976, 2020, 2):
    results[year]['results'] = get_pct_results(all_data, year)

In [ ]:
def get_total_votes_by_party(data, party, year):
    rows = data[(data['year'] == year) & (all_data['party'] == party)]
    return rows['candidatevotes'].sum()

def get_total_votes(data, year):
    rows = data[(data['year'] == year)]
    return rows['candidatevotes'].sum()

In [ ]:
get_total_votes(all_data, 2018)

113688301

In [ ]:
for year in range(1976, 2020, 2):
    results[year]['total_dem'] = get_total_votes_by_party(all_data, 'democrat', year)
    results[year]['total_rep'] = get_total_votes_by_party(all_data, 'republican', year)
    results[year]['total'] = get_total_votes(all_data, year)

In [ ]:
import sys
!{sys.executable} -m pip install plotly

from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=True)

In [ ]:
from collections import Counter

by_pct = Counter()
for row in results[2018]['results'][0].iterrows():
    by_pct[int(row[1]['rep_margin'] - national_rep_margin_2018)] += 1

In [ ]:
from collections import OrderedDict

# Note: Not Normalized
def smooth(dist, width = 4):
    print(width)
    def get_pos(key):
        total = dist[key]
        for i in range(1, width):
            total += dist.get(key - i, 0) / ( 1 ** i)
            total += dist.get(key + i, 0) / ( 1 ** i)
        return total / (2 * width + 1)
    return {key: get_pos(key) for key in dist.keys()}


def normalize_weights(weights, normalized_sum = 1):
    total = sum(weights.values())
    return {key: normalized_sum * val / total for key, val in weights.items()}

In [ ]:
smoothed_by_pct = normalize_weights(smooth(by_pct, 5), 435)
sorted_by_pct = OrderedDict(sorted(smoothed_by_pct.items()))

5


In [ ]:
trace0 = Scatter(x = list(sorted_by_pct.keys()), y = list(sorted_by_pct.values()))
data = Data([trace0])
iplot(data)

In [ ]:
cdf = {}
cur_total = 0
for key, val in sorted_by_pct.items():
    cur_total += val
    cdf[key] = cur_total

In [ ]:
trace0 = Scatter(x = list(cdf.keys()), y = list(cdf.values()))
data = Data([trace0])
iplot(data)

In [ ]:
national_rep_margin_2018 = 100 * (results[2018]['total_rep'] - results[2018]['total_dem']) / results[2018]['total']

In [ ]:
national_rep_margin_2018

-7.183550926669227

In [ ]:
rep_states = ["Idaho",
"Nevada",
"Utah",
"Arizona",
"Alaska",
"Wyoming",
"New Mexico",
"Texas",
"Oklahoma",
"Nebraska",
"Kansas",
"South Dakota",
"Iowa",
"Wisconsin",
"Michigan",
"Ohio",
"Tennessee",
"Alabama",
"Georgia",
"South Carolina",
"Florida",
"Pennsylvania",
"Maine",
"New Jersey",
"Virginia",
"North Dakota",
"Indiana",
"Mississippi",
"Louisiana",
"North Carolina"
]

# results_2018 = results[2018]['results']
# results_2018_df = results_2018[0]
# results_2018_df.reset_index(drop=True, inplace=True)
# results_2018_df
rep_data = all_data[all_data['state'].isin(rep_states)]

In [ ]:
from collections import defaultdict
from collections import Counter

rep_state_results = defaultdict(dict)
for year in range(1976, 2020, 2):
    rep_state_results[year]['results'] = get_pct_results(rep_data, year)
    rep_state_results[year]['total_dem'] = get_total_votes_by_party(rep_data, 'democrat', year)
    rep_state_results[year]['total_rep'] = get_total_votes_by_party(rep_data, 'republican', year)
    rep_state_results[year]['total'] = get_total_votes(rep_data, year)

/usr/local/Cellar/jupyterlab/2.0.1/libexec/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [ ]:
import math
by_pct = Counter()
years = [2018]
national_rep_margin_2018 = 100 * (results[2018]['total_rep'] - results[2018]['total_dem']) / results[2018]['total']

for year in years:
    for row in rep_state_results[year]['results'][0].iterrows():
        national_rep_margin = 100 * (results[year]['total_rep'] - results[year]['total_dem']) / results[year]['total']
        by_pct[int(row[1]['rep_margin'])] += 1
    
smoothed_by_pct = normalize_weights(smooth(by_pct, 3), sum(by_pct.values()))
sorted_by_pct = OrderedDict(sorted(smoothed_by_pct.items()))
dem_blowout = sum([val for key, val in sorted_by_pct.items() if key < -60])
rep_blowout = sum([val for key, val in sorted_by_pct.items() if key > 60])
print("Dem Blowout: {}\tRep Blowout: {}".format(dem_blowout, rep_blowout))
sorted_by_pct = {key: val for key, val in sorted_by_pct.items() if abs(key) <= 60}

trace0 = Scatter(x = list(sorted_by_pct.keys()), y = list(sorted_by_pct.values()))
data = Data([trace0])
iplot(data)

3
Dem Blowout: 10.75445173383317	Rep Blowout: 1.6729147141518266


In [ ]:
cdf = {}
cur_total = 0
for key, val in sorted_by_pct.items():
    cur_total += val
    cdf[key] = cur_total
    
trace0 = Scatter(x = list(cdf.keys()), y = list(cdf.values()))
data = Data([trace0])
iplot(data)

In [ ]:
dem_data = all_data[~all_data['state'].isin(rep_states)]
from collections import defaultdict
from collections import Counter

dem_state_results = defaultdict(dict)
for year in range(1976, 2020, 2):
    dem_state_results[year]['results'] = get_pct_results(dem_data, year)
    dem_state_results[year]['total_dem'] = get_total_votes_by_party(dem_data, 'democrat', year)
    dem_state_results[year]['total_rep'] = get_total_votes_by_party(dem_data, 'republican', year)
    dem_state_results[year]['total'] = get_total_votes(dem_data, year)

/usr/local/Cellar/jupyterlab/2.0.1/libexec/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [ ]:
by_pct = Counter()
year = 2014
for row in dem_state_results[year]['results'][0].iterrows():
    national_dem_margin = 100 * (results[year]['total_dem'] - results[year]['total_rep']) / results[year]['total']
    dem_state_dem_margin = 100 * (dem_state_results[year]['total_dem'] - dem_state_results[year]['total_rep']) / dem_state_results[year]['total']
    by_pct[int(row[1]['rep_margin'] + (dem_state_dem_margin - national_dem_margin))] += 1
    
smoothed_by_pct = normalize_weights(smooth(by_pct, 10), sum(by_pct.values()))
sorted_by_pct = OrderedDict(sorted(smoothed_by_pct.items()))

trace0 = Scatter(x = list(sorted_by_pct.keys()), y = list(sorted_by_pct.values()))
data = Data([trace0])
iplot(data)

10


In [ ]:
cdf = {}
cur_total = 0
for key, val in sorted_by_pct.items():
    cur_total += val
    cdf[key] = cur_total
    
trace1 = Scatter(x = list(cdf.keys()), y = list(cdf.values()))
data = Data([trace1])
iplot(data)

In [ ]:
pd.set_option('display.max_rows', 100)
pd.set_option("max_rows", None)
rep_state_results[2018]['results'][0].sort_values(by='rep_margin')

dem_pct     rep_pct  rep_margin
year state          district                                    
2018 Florida        24        100.000000    0.000000 -100.000000
                    14        100.000000    0.000000 -100.000000
                    10        100.000000    0.000000 -100.000000
     Georgia        5         100.000000    0.000000 -100.000000
     Florida        21        100.000000    0.000000 -100.000000
     Pennsylvania   18        100.000000    0.000000 -100.000000
     Florida        20         99.918649    0.000000  -99.918649
     Alabama        7          97.804539    0.000000  -97.804539
     Wisconsin      2          97.422273    0.000000  -97.422273
     Virginia       3          91.224130    0.000000  -91.224130
     Texas          30         91.052224    0.000000  -91.052224
                    9          89.055627    0.000000  -89.055627
     Pennsylvania   3          93.380780    6.619220  -86.761559
     Arizona        7          85.606319    0.000000  -85.606319
     Texas          28         84.390847    0.000000  -84.390847
     Michigan       13         84.236293    0.000000  -84.236293
     Texas          20         80.853439    0.000000  -80.853439
     Louisiana      2          80.591740    0.000000  -80.591740
     New Jersey     10         87.556892   10.087480  -77.469412
     Mississippi    2          71.786845    0.000000  -71.786845
     Ohio           11         82.238091   17.747546  -64.490545
     Michigan       14         80.882586   17.320770  -63.561816
     Tennessee      9          79.976967   19.231744  -60.745223
     New Jersey     8          78.121273   18.718843  -59.402431
     Pennsylvania   2          79.016942   20.983058  -58.033884
     Georgia        4          78.846920   21.153080  -57.693839
     Texas          18         75.246837   20.814617  -54.432220
                    33         76.163356   21.908341  -54.255016
     Wisconsin      4          75.612168   21.638159  -53.974008
     Virginia       8          76.101162   23.679592  -52.421570
     Georgia        13         76.184380   23.815620  -52.368760
     Texas          29         75.057450   23.914413  -51.143037
     Ohio           3          73.608403   26.366463  -47.241940
     Texas          35         71.252712   26.049251  -45.203461
     Virginia       11         71.108191   26.933658  -44.174534
     South Carolina 6          70.126869   28.232889  -41.893980
     Texas          16         68.464518   27.029391  -41.435127
     New Jersey     9          70.269839   28.866957  -41.402882
     Michigan       12         68.081784   28.888972  -39.192813
     New Jersey     12         68.681129   31.318871  -37.362259
     Tennessee      5          67.845322   32.151627  -35.693695
     Ohio           9          67.785218   32.194087  -35.591130
     Nevada         1          66.167108   30.865763  -35.301345
     Florida        5          66.781220   33.218780  -33.562439
     New Mexico     3          63.375025   31.208324  -32.166701
     New Jersey     1          64.394992   33.261584  -31.133408
     Pennsylvania   5          65.188669   34.811331  -30.377337
     Indiana        1          65.091820   34.906549  -30.185271
                    7          64.862865   35.137135  -29.725730
     Arizona        3          63.865462   36.134538  -27.730924
     New Jersey     6          63.632541   36.367459  -27.265083
     Pennsylvania   4          63.522438   36.477562  -27.044875
     Virginia       4          62.577788   35.919481  -26.658307
     Maine          1          58.819832   32.506074  -26.313758
     Florida        22         62.023696   37.976304  -24.047393
     Michigan       5          59.513120   35.911842  -23.601278
                    9          59.670413   36.814387  -22.856027
     New Mexico     1          59.132613   36.324560  -22.808053
     Florida        23         58.477164   35.983442  -22.493722
     Arizona        9          61.085571   38.914429  -22.171142
   

In [ ]:
dem_state_results[2018]['results'][0].sort_values(by='rep_margin')

dem_pct     rep_pct  rep_margin
year state          district                                    
2018 Washington     9         100.000000    0.000000 -100.000000
     California     44        100.000000    0.000000 -100.000000
                    6         100.000000    0.000000 -100.000000
                    27        100.000000    0.000000 -100.000000
     New York       5          99.398654    0.000000  -99.398654
     Massachusetts  8          98.424653    0.000000  -98.424653
                    7          98.252356    0.000000  -98.252356
                    4          97.718472    0.000000  -97.718472
                    1          97.644076    0.000000  -97.644076
     New York       16         94.251074    0.000000  -94.251074
                    15         94.031645    3.519048  -90.512597
                    8          89.185507    0.000000  -89.185507
     California     13         88.381038    0.000000  -88.381038
     New York       7          85.265922    0.000000  -85.265922
                    13         89.854107    5.000315  -84.853793
                    6          84.501827    0.000000  -84.501827
                    17         82.502154    0.000000  -82.502154
     California     20         81.373465    0.000000  -81.373465
                    5          78.868124    0.000000  -78.868124
                    37         89.075548   10.924452  -78.151097
                    40         77.347693    0.000000  -77.347693
     Illinois       7          87.615079   12.384921  -75.230159
     California     12         86.823182   13.176818  -73.646364
     Illinois       4          86.585153   13.414847  -73.170306
     New York       9          82.227182    9.193651  -73.033531
     California     34         72.541572    0.000000  -72.541572
     Washington     2          71.290188    0.000000  -71.290188
     New York       12         81.729574   12.087646  -69.641928
     Washington     7          83.561154   16.438846  -67.122309
     New York       14         78.126682   13.598765  -64.527917
     Missouri       1          80.102415   16.716902  -63.385513
     Illinois       2          81.055553   18.944447  -62.111107
     California     29         80.609857   19.390143  -61.219714
     New York       10         76.854635   15.970952  -60.883683
     California     14         79.222556   20.777444  -58.445112
     Maryland       4          78.054828   19.854942  -58.199886
     California     28         78.372002   21.627998  -56.744004
                    43         77.669190   22.330810  -55.338380
     Maryland       7          76.440255   21.255714  -55.184542
     Hawaii         2          77.362319   22.637681  -54.724638
     California     2          77.007955   22.992045  -54.015910
     Illinois       1          73.505632   19.760746  -53.744886
                    5          76.663645   23.334563  -53.329082
     Oregon         3          72.599564   19.823535  -52.776029
     Massachusetts  5          75.883324   24.044404  -51.838920
     Colorado       1          73.814219   23.048043  -50.766176
     California     17         75.347364   24.652636  -50.694727
     Hawaii         1          73.095526   23.070284  -50.025243
     California     18         74.491626   25.508374  -48.983252
     North Carolina 4          72.368140   24.033767  -48.334373
     California     11         74.132421   25.867579  -48.264842
                    19         73.754874   26.245126  -47.509747
     Illinois       3          73.008588   25.918579  -47.090009
                    9          73.486229   26.513771  -46.972457
     California     30         73.401585   26.598415  -46.803171
     North Carolina 12         73.072817   26.927183  -46.145634
     California     15         72.967544   27.032456  -45.935088
     Vermont        0          69.204004   25.951455  -43.252548
     Maryland       5          70.279315   27.073821  -43.205494
     California     51         71.200952   28.799048  -42.401903
   